<a href="https://colab.research.google.com/github/liuyao12/Ranger-Mish-ImageWoof-5/blob/master/Woof_128_ResNet_with_a_Twist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with a Twist

> ConvTwist + Ranger + Mish + MaxBlurPool + restrick

# setup and imports

In [ ]:
# pip install git+https://github.com/kornia/kornia

In [ ]:
pip install git+https://github.com/ayasyrev/model_constructor

  Cloning https://github.com/ayasyrev/model_constructor to /tmp/pip-req-build-hww7snva
  Running command git clone -q https://github.com/ayasyrev/model_constructor /tmp/pip-req-build-hww7snva
  Created wheel for model-constructor: filename=model_constructor-0.1.1-cp36-none-any.whl size=23583 sha256=8d42461e898e484333fe2692c2dff03a18a4605e58fe45819825a2ee33370495
  Stored in directory: /tmp/pip-ephem-wheel-cache-ztlm9xnr/wheels/5b/92/65/8093a307d5802f41f4a8776b40bc12b558e75a2a906ae8b683
Successfully built model-constructor


In [ ]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

  Cloning https://github.com/ayasyrev/imagenette_experiments to /tmp/pip-req-build-5ho30fqu
  Running command git clone -q https://github.com/ayasyrev/imagenette_experiments /tmp/pip-req-build-5ho30fqu
     |████████████████████████████████| 163kB 2.8MB/s 
  Created wheel for imagenette-experiments: filename=imagenette_experiments-0.0.1-cp36-none-any.whl size=15332 sha256=d66e5b54dcd2ab6837ef0f11c6c4fff2453ae9c7422c647026330aa33cadc813
  Stored in directory: /tmp/pip-ephem-wheel-cache-okqmrexv/wheels/af/99/98/2212941f45a18bf6d191f56c39e3569138414560c76defc0d4
Successfully built imagenette-experiments
ERROR: kornia 0.3.1 has requirement torch==1.5.0, but you'll have torch 1.5.1+cu101 which is incompatible.


In [ ]:
from imagenette_experiments.train_utils import *

In [ ]:
from kornia.contrib import MaxBlurPool2d

In [ ]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# Twist

In [30]:
class ConvTwist(nn.Module):  # replacing 3x3 Conv2d with in_channels=out_channels
    def __init__(self, ni, nf):
        super(ConvTwist, self).__init__()
        self.twist = True and ni<=256
        self.conv = nn.Conv2d(ni, nf, kernel_size=3, padding=1, bias=False, groups=1)
        if self.twist:
            Dx = torch.Tensor([[-1,0,1],[-2,0,2],[-1,0,1]]).view(1,1,3,3)
            Dy = torch.Tensor([[1,2,1],[0,0,0],[-1,-2,-1]]).view(1,1,3,3)
            self.DD = torch.cat([Dx,Dy]*ni, dim=0)
            a = 2
            self.expansion = a
            self.conv1 = nn.Conv2d(ni, ni*a, kernel_size=3, padding=1, bias=False, groups=ni)
            self.conv2 = nn.Conv2d(ni*a, ni*a*2, kernel_size=1, bias=False, groups=ni*a)
            self.conv3 = nn.Conv2d(ni*a*2, nf*2, kernel_size=1, bias=False, groups=1)
            self.XY = None
        else:
            self.conv = nn.Conv2d(ni, nf, kernel_size=3, padding=1, bias=False, groups=1)

    def forward(self, x):
        N,C,H,W = x.size()
        # out = x
        if self.twist:
            CC = C*self.expansion
            x_double = x.unsqueeze(2).expand(N,C,2,H,W).reshape(N,C*2,H,W)
            if self.XY is None:
                XX = torch.from_numpy(np.indices((1,1,H,W))[3]*2/W-1)
                YY = torch.from_numpy(np.indices((1,1,H,W))[2]*2/H-1)
                self.XY = torch.cat([XX,YY]*C, dim=1).type(x.dtype).to(x.device)
                print(self.XY.size())
                self.DD = self.DD.to(x.device)
            x_plus = F.relu(x_double)+0.01
            total = x_plus.sum((2,3))
            XYbar = (self.XY*x_plus).sum((2,3))/total
            XY = self.XY-XYbar[:,:,None,None]
            # XY = XY.unsqueeze(2).expand(N,CC,2,H,W).reshape(N,CC*2,H,W)
            XY = XY.view(N,C,2,H,W)
            XY = torch.cat([XY,XY], dim=2).view(N,C*4,H,W)
            # out = F.conv2d(x, self.DD, padding=1, groups=CC)
            out = self.conv1(x)
            out = self.conv2(out)
            out = self.conv3(XY*out)
            N,CC,H,W = out.size()
            out = out.view(N,CC//2,2,H,W).sum(2)
            out = self.conv(x) + out
        else:
            out = self.conv(x)
        return out

In [ ]:
A = torch.arange(12).view(3,4)
print(A.size())
A.unsqueeze_(1)
print(A.size())
A = A.expand((3,2,4)).reshape(6,4)
print(A)

torch.Size([3, 4])
torch.Size([3, 1, 4])
tensor([[ 0,  1,  2,  3],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 8,  9, 10, 11]])


# ResBlock

In [31]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3:  layers = [('ConvTwist', ConvTwist(ni, nf))]
        else: layers = [('Conv{}x{}'.format(ks,ks), 
                  nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [32]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, zero_bn=True):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*2, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_2", conv_layer(nh*2, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [33]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
# model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True
res = []

# Runs and results



In [34]:
epochs = [5]
for e in epochs:
    mixup=0 if e<20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.865524,1.680742,0.478748,0.904047,02:50
1,1.606406,1.507722,0.562230,0.929753,02:49
2,1.455848,1.348176,0.635785,0.949860,02:49
3,1.294118,1.195693,0.716976,0.966149,02:49
4,1.112932,1.039041,0.794604,0.974039,02:49


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
[0.794604] 0.7946042418479919


In [ ]:
epochs = [80]
for e in epochs:
    mixup=0 if e<20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 128, 48, 48])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 256, 24, 24])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])
torch.Size([1, 512, 12, 12])


In [ ]:
res

NameError: ignored

conv: fully, conv1x1: fully

in 5, 5, 5, 20, and 80 epochs:

`[0.795113, 0.78646, 0.792314, 0.885722, 0.899211]`

In [ ]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))